Check https://github.com/vanandrew/brainextractor for documentation.


In [1]:
!pip install brainextractor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.5/688.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.3/858.3 kB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745193 sha256=fe7785d156dbd5fb4636035250cf4789b203e02141a0f1a8fcd2ab7d48d0b348
  Stored in directory: /root/.cache/pip/wheels/a1/3c/d2/1f9533f908d86176637521e533c6cdb2d4e48b59003b5c3f19
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.7
    Uninstalling PyOpenGL-3.1.7:
      Successfully uninstalled PyOpenGL-3.1.7


In [2]:
#mount drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#specify file path
file_path = "/content/drive/MyDrive/NFBS/NFBS_Dataset/A00034854/sub-A00034854_ses-NFB3_T1w.nii.gz"

In [4]:
# import the nibabel library so we can read in a nifti image
import nibabel as nib
# import the BrainExtractor class
from brainextractor import BrainExtractor

# read in the image file first
input_img = nib.load(file_path)

# create a BrainExtractor object using the input_img as input
# we just use the default arguments here, but look at the
# BrainExtractor class in the code for the full argument list
bet = BrainExtractor(img=input_img)

# run the brain extraction
# this will by default run for 1000 iterations
# I recommend looking at the run method to see how it works
bet.run()

# save the computed mask out to file
bet.save_mask("/content/MNI_mask.nii.gz")

Initializing...
Parameters: bt=0.500000, d1=20.000000, d2=10.000000, rmin=3.330000, rmax=10.000000
tmin: 0.000000, t2: 0.000000, t: 40.500000, t98: 405.000000, tmax: 966.000000
Center-of-Mass: [157.43777283 144.72866493  98.2966659 ]
Head Radius: 93.745864
Median within Head Radius: 123.000000
Initializing surface...
Brain extractor initialization complete!
Running surface deformation...
Iteration: 999
Complete.


In [5]:
mask_path = "/content/MNI_mask.nii.gz"

In [7]:
#display mask
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import numpy as np

def plot_slice(slice_number):
    # Display the selected slice
    plt.imshow(image_data[:, :, slice_number], cmap='gray')
    plt.title(f'Slice number: {slice_number}')
    plt.axis('off')
    plt.show()

def load_and_visualize_nii(file_path):
    # Load the .nii.gz file
    global image_data
    nii_data = nib.load(file_path)

    # Get the image data as a numpy array
    image_data = nii_data.get_fdata()

    # Create a slider for slice selection
    max_slice = image_data.shape[2] - 1
    interact(plot_slice, slice_number=IntSlider(min=0, max=max_slice, step=1, value=max_slice//2))


load_and_visualize_nii(mask_path)


interactive(children=(IntSlider(value=95, description='slice_number', max=191), Output()), _dom_classes=('widg…

In [8]:
#display brainn slides

def plot_slice(slice_number):
    # Display the selected slice
    plt.imshow(image_data[:, :, slice_number], cmap='gray')
    plt.title(f'Slice number: {slice_number}')
    plt.axis('off')
    plt.show()

def load_and_visualize_nii(file_path):
    # Load the .nii.gz file
    global image_data
    nii_data = nib.load(file_path)

    # Get the image data as a numpy array
    image_data = nii_data.get_fdata()

    # Create a slider for slice selection
    max_slice = image_data.shape[2] - 1
    interact(plot_slice, slice_number=IntSlider(min=0, max=max_slice, step=1, value=max_slice//2))

# Upload your .nii.gz file to Colab and replace 'your_file_path.nii.gz' with the path to your file
load_and_visualize_nii(file_path)


interactive(children=(IntSlider(value=95, description='slice_number', max=191), Output()), _dom_classes=('widg…

In [9]:
#apply mask

def apply_mask(scan_path, mask_path, output_path):
    # Load the scan and the mask
    scan_nii = nib.load(file_path)
    mask_nii = nib.load(mask_path)

    # Get data from both
    scan_data = scan_nii.get_fdata()
    mask_data = mask_nii.get_fdata()

    # Ensure the mask and scan are the same shape
    if scan_data.shape != mask_data.shape:
        raise ValueError("Scan and mask must be the same shape")

    # Apply the mask: Set voxels to zero where mask is zero
    scan_data[mask_data == 0] = 0

    # Create a new NIfTI image from the masked data
    new_scan_nii = nib.Nifti1Image(scan_data, scan_nii.affine)

    # Save the new NIfTI file
    nib.save(new_scan_nii, output_path)

# Replace these paths with your actual file paths
scan_path = file_path
mask_path = mask_path
output_path = "./output_masked_scan.nii.gz"

apply_mask(scan_path, mask_path, output_path)


In [10]:
#display segmented brainslices

def plot_slice(slice_number):
    # Display the selected slice
    plt.imshow(image_data[:, :, slice_number], cmap='gray')
    plt.title(f'Slice number: {slice_number}')
    plt.axis('off')
    plt.show()

def load_and_visualize_nii(file_path):
    # Load the .nii.gz file
    global image_data
    nii_data = nib.load(file_path)

    # Get the image data as a numpy array
    image_data = nii_data.get_fdata()

    # Create a slider for slice selection
    max_slice = image_data.shape[2] - 1
    interact(plot_slice, slice_number=IntSlider(min=0, max=max_slice, step=1, value=max_slice//2))

# Upload your .nii.gz file to Colab and replace 'your_file_path.nii.gz' with the path to your file
load_and_visualize_nii("/content/output_masked_scan.nii.gz")


interactive(children=(IntSlider(value=95, description='slice_number', max=191), Output()), _dom_classes=('widg…